In [2]:
pip install aif360

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install BlackBoxAuditing

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
from PIL import Image
import numpy as np
import pandas as pd
import os
import torch
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from glob import glob
from torch import nn, optim
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [4]:
import pandas as pd
import aif360
from aif360 import algorithms
from aif360.algorithms import preprocessing

In [5]:
labels = pd.read_csv("/home/glab/Desktop/kaggle/labels.csv")

In [6]:
labels

,Female,Young,Attractive,Smiling,Oval_Face,Wavy_Hair,path
0,1.0,1.0,1.0,1.0,0.0,1.0,000000_64.jpg
1,0.0,1.0,0.0,1.0,1.0,0.0,000001_64.jpg
2,1.0,0.0,1.0,1.0,0.0,0.0,000002_64.jpg
3,1.0,1.0,1.0,1.0,1.0,0.0,000003_64.jpg
4,0.0,1.0,0.0,0.0,0.0,0.0,000004_64.jpg
...,...,...,...,...,...,...,...
162765,0.0,1.0,1.0,1.0,1.0,0.0,162765_64.jpg
162766,0.0,1.0,0.0,0.0,0.0,1.0,162766_64.jpg
162767,1.0,1.0,1.0,1.0,1.0,1.0,162767_64.jpg
162768,1.0,1.0,1.0,0.0,1.0,1.0,162768_64.jpg


In [11]:
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, labels, root):
        self.img_labels = labels
        self.root = root

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, index):
        img_path = self.root + self.img_labels['path'].iloc[index]
        image = open_image(img_path, numpy_format=True)
        label = self.img_labels.iloc[index, :6].to_numpy().astype('float')
        # Convert image and label to torch tensors
        image = torch.from_numpy(image)
        label = torch.from_numpy(label).float()
        # Pytorch works with C x H x W format
        image = image.permute(2, 0, 1)
        # Normalize the image
        image = image/255

        return image, label

In [7]:
def disparate_impact_fem(labels):
    numerator = (labels[:, 2] == 1) & (labels[:, 0] == 1)
    denominator = (labels[:, 2] == 1) & (labels[:, 0] == 0)
    return numerator.sum() / (denominator.sum() + 1e-8)

def disparate_impact_youn(labels):
    numerator = (labels[:, 2] == 1) & (labels[:, 1] == 1)
    denominator = (labels[:, 2] == 1) & (labels[:, 1] == 0)
    return numerator.sum() / (denominator.sum() + 1e-8)

In [8]:
disparate_impact_fem(labels)

InvalidIndexError: (slice(None, None, None), 2)

In [7]:
disparate_impact_youn(labels)

14.58594332584902

In [23]:
type(labels)

pandas.core.frame.DataFrame

In [22]:
new_labels = aif360.algorithms.preprocessing.DisparateImpactRemover(repair_level=1.0, sensitive_attribute='Attractive').fit_transform(labels)

AttributeError: 'DataFrame' object has no attribute 'features'

In [13]:
new_labels

In [17]:
disparate_impact_youn_(new_labels)

TypeError: 'DisparateImpactRemover' object is not subscriptable